# 표정 특징 추출 모델

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1536461529392122462
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 locality {
   bus_id: 1
 }
 incarnation: 3162032233914724620
 physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
 xla_global_id: -1]

### 모델 정의

In [3]:
feature_dim = 400    # 특징 차원 수에 맞게 변경

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(feature_dim)
])

# 모델 컴파일
model.compile(optimizer='adam')

# 모델 요약 출력
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

## 특징 추출 완성!

In [ ]:
from keras.models import save_model
network_saved = save_model(model, 'model/featureAI/weights.hdf5')

In [2]:
with open('model/featureAI/featureAI.json', 'r') as json_file:
    json_saved_model = json_file.read()
json_saved_model

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 64, 64, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "conv2d_input"}}, {"class_name": "Conv2D", "config": {"name": "conv2d", "trainable": true, "batch_input_shape": [null, 64, 64, 3], "dtype": "float32", "filters": 32, "kernel_size": [3, 3], "strides": [1, 1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "groups": 1, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "MaxPooling2D", "config": {"name": "max_pooling2d", "trainable": true, "dtype": "float32", "pool_size": [2, 2], "padding": "valid", "strides": [2, 2], "d

In [5]:
network_loaded = tf.keras.models.model_from_json(json_saved_model)
network_loaded.load_weights('model/featureAI/weights.hdf5')
network_loaded.compile(optimizer='Adam')
network_loaded.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

In [42]:
# 이미지 50개만 선택해서 특징 추출

import os
import cv2

happyList = os.listdir('한국인 감정인식을 위한 복합 영상/Training1/기쁨')

prediction = []
for i in range(50):
    image = cv2.imread('한국인 감정인식을 위한 복합 영상/Training1/기쁨/' + happyList[i])
    roi = cv2.resize(image, (64, 64))
    roi = roi / 255
    
    feature = model.predict(np.expand_dims(roi, axis=0))

    prediction.append(feature)

1/1 [==============================] - 0s 11ms/step


In [45]:
prediction[0]

array([[ 0.14744748,  0.01604744,  0.08844653,  0.02901251,  0.05955495,
        -0.01233516, -0.02983252, -0.02208188, -0.04033911,  0.05752836,
        -0.1147261 , -0.00025791, -0.08122497, -0.07894939, -0.0045892 ,
         0.0151031 ,  0.03327703,  0.006045  , -0.03690238, -0.01442665,
         0.03890117,  0.00524338, -0.01636473,  0.01741934, -0.06158079,
        -0.11296864,  0.03223491, -0.00230795,  0.00798451, -0.02755954,
         0.02555803,  0.14370099, -0.00533314,  0.05907374,  0.12331662,
         0.03378845,  0.03845372,  0.06595925, -0.05906901, -0.07947154,
         0.01964581, -0.01703541,  0.03860863,  0.0515989 ,  0.0247642 ,
        -0.00204735, -0.0198562 , -0.04539012, -0.06064746, -0.02237985,
         0.02845267,  0.0597506 , -0.0294268 , -0.00108065,  0.05312102,
        -0.00988549, -0.08298852, -0.01707399,  0.00327575, -0.06556924,
         0.00676116,  0.06503423, -0.05987858, -0.07182973,  0.04641211,
        -0.04787118, -0.02736631,  0.1621202 ,  0.0